Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (448297, 28, 28) (448297,)
Validation set (13649, 28, 28) (13649,)
Test set (13649, 28, 28) (13649,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (448297, 28, 28, 1) (448297, 10)
Validation set (13649, 28, 28, 1) (13649, 10)
Test set (13649, 28, 28, 1) (13649, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [16]:
%%time
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.929373
Minibatch accuracy: 6.2%
Validation accuracy: 11.4%
Minibatch loss at step 500: 0.808260
Minibatch accuracy: 75.0%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 0.891980
Minibatch accuracy: 75.0%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 0.520008
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 0.965271
Minibatch accuracy: 75.0%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.372357
Minibatch accuracy: 87.5%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 0.197823
Minibatch accuracy: 93.8%
Validation accuracy: 84.7%
Minibatch loss at step 3500: 0.283527
Minibatch accuracy: 100.0%
Validation accuracy: 85.2%
Minibatch loss at step 4000: 2.145947
Minibatch accuracy: 62.5%
Validation accuracy: 84.2%
Minibatch loss at step 4500: 0.021469
Minibatch accuracy: 100.0%
Validation accuracy: 85.5%
Minibatch loss at step 5000: 0.683975
Minibatch accuracy: 75.0%
Validation accura

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [61]:
%%time
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(conv,[1,2,2,1],[1,2,2,1],padding='SAME')
    hidden = tf.nn.relu(hidden + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(conv,[1,2,2,1],[1,2,2,1],padding='SAME')    
    hidden = tf.nn.relu(hidden + layer2_biases)
    shape = hidden.get_shape().as_list()
    print (hidden)    
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Tensor("Relu_1:0", shape=(16, 7, 7, 16), dtype=float32)
Tensor("Reshape:0", shape=(16, 784), dtype=float32)
Tensor("Relu_4:0", shape=(13649, 7, 7, 16), dtype=float32)
Tensor("Reshape_4:0", shape=(13649, 784), dtype=float32)
Tensor("Relu_7:0", shape=(13649, 7, 7, 16), dtype=float32)
Tensor("Reshape_5:0", shape=(13649, 784), dtype=float32)
Initialized
Minibatch loss at step 0: 3.504916
Minibatch accuracy: 12.5%
Validation accuracy: 9.9%
Minibatch loss at step 50: 2.216789
Minibatch accuracy: 50.0%
Validation accuracy: 21.1%
Minibatch loss at step 100: 1.331061
Minibatch accuracy: 56.2%
Validation accuracy: 48.7%
Minibatch loss at step 150: 1.107399
Minibatch accuracy: 68.8%
Validation accuracy: 65.7%
Minibatch loss at step 200: 0.852027
Minibatch accuracy: 75.0%
Validation accuracy: 63.2%
Minibatch loss at step 250: 1.041991
Minibatch accuracy: 81.2%
Validation accuracy: 68.7%
Minibatch loss at step 300: 0.740036
Minibatch accuracy: 68.8%
Validation accuracy: 71.3%
Minibatch loss at step

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### 94.6% in 10k steps.

In [91]:
%%time
batch_size = 16
patch_size = 5
depth = 28
num_hidden = 512

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden/2], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden/2]))

  layer5_weights = tf.Variable(tf.truncated_normal([num_hidden/2, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(conv,[1,2,2,1],[1,2,2,1],padding='SAME')
    hidden = tf.nn.relu(hidden + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(conv,[1,2,2,1],[1,2,2,1],padding='SAME')    
    hidden = tf.nn.relu(hidden + layer2_biases)
    shape = hidden.get_shape().as_list()
    print (hidden)    
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden = tf.nn.dropout(hidden,keep_prob=0.8)
    hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
    
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.AdamOptimizer().minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Tensor("Relu_1:0", shape=(32, 7, 7, 28), dtype=float32)
Tensor("Reshape:0", shape=(32, 1372), dtype=float32)
Tensor("Relu_5:0", shape=(13649, 7, 7, 28), dtype=float32)
Tensor("Reshape_4:0", shape=(13649, 1372), dtype=float32)
Tensor("Relu_9:0", shape=(13649, 7, 7, 28), dtype=float32)
Tensor("Reshape_5:0", shape=(13649, 1372), dtype=float32)
Initialized
Minibatch loss at step 0: 23.314960
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 50: 1.299581
Minibatch accuracy: 56.2%
Validation accuracy: 55.6%
Minibatch loss at step 100: 1.025456
Minibatch accuracy: 75.0%
Validation accuracy: 70.2%
Minibatch loss at step 150: 0.679961
Minibatch accuracy: 78.1%
Validation accuracy: 72.5%
Minibatch loss at step 200: 0.350865
Minibatch accuracy: 87.5%
Validation accuracy: 74.2%
Minibatch loss at step 250: 0.739173
Minibatch accuracy: 71.9%
Validation accuracy: 74.8%
Minibatch loss at step 300: 0.672593
Minibatch accuracy: 81.2%
Validation accuracy: 77.9%
Minibatch loss at

# Copying the best ideas from an example

In [55]:
%%time
batch_size = 16
patch_size = 3
depth = 16
num_hidden = 256
num_hidden_last = num_hidden/4

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    tf_keep_prob = tf.placeholder(tf.float32)

    # Variables.
    conv1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    conv1_biases = tf.Variable(tf.zeros([depth]))
    
    conv2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth*2], stddev=0.1))
    conv2_biases = tf.Variable(tf.zeros([depth*2]))
    
    conv3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth*2, depth*4], stddev=0.03))
    conv3_biases = tf.Variable(tf.zeros([depth*4]))
    
    conv4_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth*4, depth*8], stddev=0.03))
    conv4_biases = tf.Variable(tf.zeros([depth*8]))
    
    conv5_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth*8, depth*16], stddev=0.03))
    conv5_biases = tf.Variable(tf.zeros([depth*16]))
    
    fc1_weights = tf.Variable(tf.truncated_normal([(image_size + 4) // 2**5 * (image_size + 4) // 2**5 * depth*16, num_hidden], stddev=0.03))
    fc1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    fc2_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden_last], stddev=np.sqrt(2.0/num_hidden)))
    fc2_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_last]))

    fc3_weights = tf.Variable(tf.truncated_normal([num_hidden_last, num_labels], stddev=np.sqrt(2.0/num_hidden_last)))
    fc3_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    def convolve(layer, weights, biases):
        conv = tf.nn.conv2d(layer, weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.elu(conv + biases)
        pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        print ('conv',
               conv.get_shape().as_list(),
               pool.get_shape().as_list(),
               weights.get_shape().as_list())
        return pool
        
    def fully_connect(layer, weights, biases, use_dropout):
        logits = tf.nn.elu(tf.matmul(layer, weights) + biases)
        if use_dropout:
            logits = tf.nn.dropout(logits, keep_prob=0.75)
        print ('fc',
               logits.get_shape().as_list(),
               weights.get_shape().as_list())
        return logits
        
    # Model.
    def model(data, use_dropout=False):
        layer = convolve(data, conv1_weights, conv1_biases)
        layer = convolve(layer, conv2_weights, conv2_biases)
        layer = convolve(layer, conv3_weights, conv3_biases)
        layer = convolve(layer, conv4_weights, conv4_biases)
        layer = convolve(layer, conv5_weights, conv5_biases)
        
        shape = layer.get_shape().as_list()
        reshaped_layer = tf.reshape(layer, [shape[0], shape[1] * shape[2] * shape[3]])
        reshape = reshaped_layer.get_shape().as_list()
        print ('reshaping',shape, reshape)
        
        layer = fully_connect(reshaped_layer, fc1_weights, fc1_biases, use_dropout)
        layer = fully_connect(layer, fc2_weights, fc2_biases, use_dropout)
        return tf.matmul(layer, fc3_weights) + fc3_biases
    
    # Training computation.
    logits = model(tf_train_dataset, use_dropout=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.1, global_step, 3000, 0.86, staircase=True)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 95001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print ("Initialized")
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print ("Minibatch loss at step", step, ":", l)
            print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print ("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print ("Learning rate: %.6f" % learning_rate.eval())
    print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

conv [16, 28, 28, 16] [16, 14, 14, 16] [3, 3, 1, 16]
conv [16, 14, 14, 32] [16, 7, 7, 32] [3, 3, 16, 32]
conv [16, 7, 7, 64] [16, 4, 4, 64] [3, 3, 32, 64]
conv [16, 4, 4, 128] [16, 2, 2, 128] [3, 3, 64, 128]
conv [16, 2, 2, 256] [16, 1, 1, 256] [3, 3, 128, 256]
reshaping [16, 1, 1, 256] [16, 256]
fc [16, 256] [256, 256]
fc [16, 64] [256, 64]
conv [13649, 28, 28, 16] [13649, 14, 14, 16] [3, 3, 1, 16]
conv [13649, 14, 14, 32] [13649, 7, 7, 32] [3, 3, 16, 32]
conv [13649, 7, 7, 64] [13649, 4, 4, 64] [3, 3, 32, 64]
conv [13649, 4, 4, 128] [13649, 2, 2, 128] [3, 3, 64, 128]
conv [13649, 2, 2, 256] [13649, 1, 1, 256] [3, 3, 128, 256]
reshaping [13649, 1, 1, 256] [13649, 256]
fc [13649, 256] [256, 256]
fc [13649, 64] [256, 64]
conv [13649, 28, 28, 16] [13649, 14, 14, 16] [3, 3, 1, 16]
conv [13649, 14, 14, 32] [13649, 7, 7, 32] [3, 3, 16, 32]
conv [13649, 7, 7, 64] [13649, 4, 4, 64] [3, 3, 32, 64]
conv [13649, 4, 4, 128] [13649, 2, 2, 128] [3, 3, 64, 128]
conv [13649, 2, 2, 256] [13649, 1, 1, 

## Example solution 1 - 97.1% after 95000 steps by [Endri Deliu](https://discussions.udacity.com/t/assignment-4-problem-2/46525/35?u=brtknr)

In [44]:
%%time
batch_size = 16
patch_size = 3
depth = 16
num_hidden = 256
num_hidden_last = 128
stride = 2

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layerconv1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layerconv1_biases = tf.Variable(tf.zeros([depth]))

  layerconv2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth * 2], stddev=0.1))
  layerconv2_biases = tf.Variable(tf.zeros([depth * 2]))
    
  layerconv3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth * 2, depth * 4], stddev=0.03))
  layerconv3_biases = tf.Variable(tf.zeros([depth * 4]))
  
  layerconv4_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth * 4, depth * 8], stddev=0.03))
  layerconv4_biases = tf.Variable(tf.zeros([depth * 8]))
  
  layerconv5_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth * 8, depth * 16], stddev=0.03))
  layerconv5_biases = tf.Variable(tf.zeros([depth * 16]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 28 * image_size // 28 * (depth*16), num_hidden], stddev=0.03))
  layer3_biases = tf.Variable(tf.zeros([num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden_last], stddev=0.0532))
  layer4_biases = tf.Variable(tf.zeros([num_hidden_last]))

  layer5_weights = tf.Variable(tf.truncated_normal([num_hidden_last, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.zeros([num_labels]))

  # Model.
  def model(data, use_dropout=False):
    conv = tf.nn.conv2d(data, layerconv1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print (conv.get_shape().as_list(),
           pool.get_shape().as_list(),
           layerconv1_weights.get_shape().as_list())
    
    conv = tf.nn.conv2d(pool, layerconv2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print (conv.get_shape().as_list(),
           pool.get_shape().as_list(),
           layerconv2_weights.get_shape().as_list())
    
    conv = tf.nn.conv2d(pool, layerconv3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv3_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print (conv.get_shape().as_list(),
           pool.get_shape().as_list(),
           layerconv3_weights.get_shape().as_list())
    # norm1
    # norm1 = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    conv = tf.nn.conv2d(pool, layerconv4_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv4_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print (conv.get_shape().as_list(),
       pool.get_shape().as_list(),
       layerconv4_weights.get_shape().as_list())
    # norm1 = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

    conv = tf.nn.conv2d(pool, layerconv5_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv5_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    print (conv.get_shape().as_list(),
           pool.get_shape().as_list(),
           layerconv5_weights.get_shape().as_list())
    # norm1 = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    shape = pool.get_shape().as_list()
    print (pool)
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    print (reshape)
    logits = tf.matmul(reshape, layer3_weights) + layer3_biases
    hidden = tf.nn.elu(logits)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    
    logits = tf.matmul(hidden, layer4_weights) + layer4_biases
    hidden = tf.nn.elu(logits)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    
    
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset, use_dropout=True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 3000, 0.86, staircase=True)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


num_steps = 95001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print ("Initialized")
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

[16, 28, 28, 16] [16, 14, 14, 16] [3, 3, 1, 16]
[16, 14, 14, 32] [16, 7, 7, 32] [3, 3, 16, 32]
[16, 7, 7, 64] [16, 4, 4, 64] [3, 3, 32, 64]
[16, 4, 4, 128] [16, 2, 2, 128] [3, 3, 64, 128]
[16, 2, 2, 256] [16, 1, 1, 256] [3, 3, 128, 256]
Tensor("MaxPool_4:0", shape=(16, 1, 1, 256), dtype=float32)
Tensor("Reshape:0", shape=(16, 256), dtype=float32)
[13649, 28, 28, 16] [13649, 14, 14, 16] [3, 3, 1, 16]
[13649, 14, 14, 32] [13649, 7, 7, 32] [3, 3, 16, 32]
[13649, 7, 7, 64] [13649, 4, 4, 64] [3, 3, 32, 64]
[13649, 4, 4, 128] [13649, 2, 2, 128] [3, 3, 64, 128]
[13649, 2, 2, 256] [13649, 1, 1, 256] [3, 3, 128, 256]
Tensor("MaxPool_9:0", shape=(13649, 1, 1, 256), dtype=float32)
Tensor("Reshape_4:0", shape=(13649, 256), dtype=float32)
[13649, 28, 28, 16] [13649, 14, 14, 16] [3, 3, 1, 16]
[13649, 14, 14, 32] [13649, 7, 7, 32] [3, 3, 16, 32]
[13649, 7, 7, 64] [13649, 4, 4, 64] [3, 3, 32, 64]
[13649, 4, 4, 128] [13649, 2, 2, 128] [3, 3, 64, 128]
[13649, 2, 2, 256] [13649, 1, 1, 256] [3, 3, 128, 25

### But 94.4% in 10k steps

In [92]:
%%time
batch_size = 16
patch_size = 3
depth = 16
num_hidden = 705
num_hidden_last = 205

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layerconv1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layerconv1_biases = tf.Variable(tf.zeros([depth]))
  layerconv2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth * 2], stddev=0.1))
  layerconv2_biases = tf.Variable(tf.zeros([depth * 2]))
  
  layerconv3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth * 2, depth * 4], stddev=0.03))
  layerconv3_biases = tf.Variable(tf.zeros([depth * 4]))
  
  layerconv4_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth * 4, depth * 4], stddev=0.03))
  layerconv4_biases = tf.Variable(tf.zeros([depth * 4]))
  

  layerconv5_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth * 4, depth * 16], stddev=0.03))
  layerconv5_biases = tf.Variable(tf.zeros([depth * 16]))

    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 7 * image_size / 7 * (depth * 4), num_hidden], stddev=0.03))
  layer3_biases = tf.Variable(tf.zeros([num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden_last], stddev=0.0532))
  layer4_biases = tf.Variable(tf.zeros([num_hidden_last]))
  
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_last, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.zeros([num_labels]))
  

  # Model.
  def model(data, use_dropout=False):
    conv = tf.nn.conv2d(data, layerconv1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layerconv2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv2_biases)
    #pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    

    conv = tf.nn.conv2d(hidden, layerconv3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv3_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    # norm1
    # norm1 = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    conv = tf.nn.conv2d(pool, layerconv4_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv4_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    # norm1 = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

    
    conv = tf.nn.conv2d(pool, layerconv5_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.elu(conv + layerconv5_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    # norm1 = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    shape = pool.get_shape().as_list()
    print (shape)
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.elu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    
    nn_hidden_layer = tf.matmul(hidden, layer4_weights) + layer4_biases
    hidden = tf.nn.elu(nn_hidden_layer)
    
    if use_dropout:
        hidden = tf.nn.dropout(hidden, 0.75)
    
    
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset, True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 3000, 0.86, staircase=True)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print ("Initialized")
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

[16, 2, 2, 256]
[13649, 2, 2, 256]
[13649, 2, 2, 256]
Initialized
Minibatch loss at step 0 : 2.29201
Minibatch accuracy: 18.8%
Validation accuracy: 11.1%
Minibatch loss at step 500 : 0.438783
Minibatch accuracy: 93.8%
Validation accuracy: 84.1%
Minibatch loss at step 1000 : 0.910752
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 1500 : 0.233955
Minibatch accuracy: 93.8%
Validation accuracy: 85.9%
Minibatch loss at step 2000 : 0.914765
Minibatch accuracy: 75.0%
Validation accuracy: 85.7%
Minibatch loss at step 2500 : 0.336413
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 3000 : 0.218305
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%
Minibatch loss at step 3500 : 0.416173
Minibatch accuracy: 81.2%
Validation accuracy: 87.7%
Minibatch loss at step 4000 : 1.59111
Minibatch accuracy: 75.0%
Validation accuracy: 87.3%
Minibatch loss at step 4500 : 0.0347821
Minibatch accuracy: 100.0%
Validation accuracy: 88.3%
Minibatch loss at 

## Example solution 2 - 94.5% in 30k steps by [Kobi Bento](https://discussions.udacity.com/t/assignment-4-problem-2/46525/54?u=brtknr)

In [75]:
batch_size = 16
num_channels = 1

c1_depth = 6
c1_ker_sz = 5
c3_depth = 16
c3_ker_sz = 6
c5_depth = 120
c5_ker_sz = 6

num_hidden = 84

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    c1_weights = tf.Variable(tf.truncated_normal([c1_ker_sz, c1_ker_sz, num_channels, c1_depth], stddev=0.1))
    c1_biases = tf.Variable(tf.zeros([c1_depth]))
    c3_weights = tf.Variable(tf.truncated_normal([c3_ker_sz, c3_ker_sz, c1_depth, c3_depth], stddev=0.1))
    c3_biases = tf.Variable(tf.constant(1.0, shape=[c3_depth]))
    c5_weights = tf.Variable(tf.truncated_normal([c5_ker_sz, c5_ker_sz, c3_depth, c5_depth], stddev=0.1))
    c5_biases = tf.Variable(tf.constant(1.0, shape=[c5_depth]))
    c5_conv_dim = (((((image_size+1)//2) + 1) // 2) + 1 )//2
    fc_weights = tf.Variable(tf.truncated_normal([c5_conv_dim * c5_conv_dim * c5_depth, num_hidden], stddev=0.1))
    fc_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    out_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    out_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        print(data.get_shape().as_list())
        conv = tf.nn.conv2d(data, c1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c1_biases)
        print(conv.get_shape().as_list())
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        print(pooled.get_shape().as_list())
        conv = tf.nn.conv2d(pooled, c3_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c3_biases)
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = pooled.get_shape().as_list()
        print(shape)
        conv = tf.nn.conv2d(pooled, c5_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c5_biases)
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = pooled.get_shape().as_list()
        print(shape)
        reshape = tf.reshape(pooled, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, fc_weights) + fc_biases)
        return tf.matmul(hidden, out_weights) + out_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    optimizer = tf.train.AdagradOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print ("Initialized")
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

[16, 28, 28, 1]
[16, 28, 28, 6]
[16, 14, 14, 6]
[16, 7, 7, 16]
[16, 4, 4, 120]
[13649, 28, 28, 1]
[13649, 28, 28, 6]
[13649, 14, 14, 6]
[13649, 7, 7, 16]
[13649, 4, 4, 120]
[13649, 28, 28, 1]
[13649, 28, 28, 6]
[13649, 14, 14, 6]
[13649, 7, 7, 16]
[13649, 4, 4, 120]
Initialized
Minibatch loss at step 0 : 12.3073
Minibatch accuracy: 6.2%
Validation accuracy: 7.3%
Minibatch loss at step 500 : 2.30312
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 1000 : 2.30001
Minibatch accuracy: 6.2%
Validation accuracy: 10.6%
Minibatch loss at step 1500 : 2.29376
Minibatch accuracy: 12.5%
Validation accuracy: 10.6%
Minibatch loss at step 2000 : 2.29285
Minibatch accuracy: 18.8%
Validation accuracy: 10.4%
Minibatch loss at step 2500 : 2.30703
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 3000 : 2.29797
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss at step 3500 : 2.2893
Minibatch accuracy: 12.5%
Validation accuracy: 10.6%
Min

### However in 10k steps...

In [93]:
batch_size = 16
num_channels = 1

c1_depth = 6
c1_ker_sz = 5
c3_depth = 16
c3_ker_sz = 6
c5_depth = 120
c5_ker_sz = 6

num_hidden = 84

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    c1_weights = tf.Variable(tf.truncated_normal([c1_ker_sz, c1_ker_sz, num_channels, c1_depth], stddev=0.1))
    c1_biases = tf.Variable(tf.zeros([c1_depth]))
    c3_weights = tf.Variable(tf.truncated_normal([c3_ker_sz, c3_ker_sz, c1_depth, c3_depth], stddev=0.1))
    c3_biases = tf.Variable(tf.constant(1.0, shape=[c3_depth]))
    c5_weights = tf.Variable(tf.truncated_normal([c5_ker_sz, c5_ker_sz, c3_depth, c5_depth], stddev=0.1))
    c5_biases = tf.Variable(tf.constant(1.0, shape=[c5_depth]))
    c5_conv_dim = (((((image_size+1)//2) + 1) // 2) + 1 )//2
    fc_weights = tf.Variable(tf.truncated_normal([c5_conv_dim * c5_conv_dim * c5_depth, num_hidden], stddev=0.1))
    fc_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    out_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    out_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        print(data.get_shape().as_list())
        conv = tf.nn.conv2d(data, c1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c1_biases)
        print(conv.get_shape().as_list())
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        print(pooled.get_shape().as_list())
        conv = tf.nn.conv2d(pooled, c3_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c3_biases)
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = pooled.get_shape().as_list()
        print(shape)
        conv = tf.nn.conv2d(pooled, c5_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c5_biases)
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = pooled.get_shape().as_list()
        print(shape)
        reshape = tf.reshape(pooled, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, fc_weights) + fc_biases)
        return tf.matmul(hidden, out_weights) + out_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    optimizer = tf.train.AdagradOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print ("Initialized")
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

[16, 28, 28, 1]
[16, 28, 28, 6]
[16, 14, 14, 6]
[16, 7, 7, 16]
[16, 4, 4, 120]
[13649, 28, 28, 1]
[13649, 28, 28, 6]
[13649, 14, 14, 6]
[13649, 7, 7, 16]
[13649, 4, 4, 120]
[13649, 28, 28, 1]
[13649, 28, 28, 6]
[13649, 14, 14, 6]
[13649, 7, 7, 16]
[13649, 4, 4, 120]
Initialized
Minibatch loss at step 0 : 8.93694
Minibatch accuracy: 6.2%
Validation accuracy: 9.9%
Minibatch loss at step 500 : 2.30322
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 1000 : 2.29927
Minibatch accuracy: 6.2%
Validation accuracy: 10.6%
Minibatch loss at step 1500 : 2.29383
Minibatch accuracy: 12.5%
Validation accuracy: 10.6%
Minibatch loss at step 2000 : 2.29308
Minibatch accuracy: 18.8%
Validation accuracy: 10.4%
Minibatch loss at step 2500 : 2.30724
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 3000 : 2.29782
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss at step 3500 : 2.28924
Minibatch accuracy: 12.5%
Validation accuracy: 10.6%
Mi